# Data Querying and Exporting

This notebook demonstrates the process of querying data from Delta Lake and exporting it in various formats.

## Starting the servers:
To launch the server, open the Docker Desktop app and open a terminal in the root of the [DiveDB repo](https://github.com/ecophysviz-lab/DiveDB). Then run the following command:
```bash
$ make up
```
This command will launch the Django server, Postgres database, and Jupyter server using the environment variables defined in the `.env` file accross all containers. 

**Ensure you have the same env variables in Pylogger**

#### Understanding expected file paths:
DiveDB expects the following paths to be set in the `.env` file:
- `CONTAINER_DATA_PATH`
- `LOCAL_DATA_PATH`
- `HOST_DELTA_LAKE_PATH`
- `CONTAINER_DELTA_LAKE_PATH`
- `HOST_FILE_STORAGE_PATH`
- `CONTAINER_FILE_STORAGE_PATH`

These paths are used to mount the Delta Lake and file storage to the containers. The "LOCAL_" and "HOST_" paths can be wherever makes sense for your local machine. The "CONTAINER_" paths are the paths that the containers expect. We recommend you keep the "CONTAINER_" paths as they are in the `.env.example` file.

#### When is the server ready?
There are 3 processes that need to be running for the server to be ready:
1. The Django server (`web`)
2. The Postgres database (`metadata_database`)
3. The Jupyter server (`jupyter`)

Jupyter is almost always the last to start up. You'll know it's ready when you see the following logs in the terminal:
```bash
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Serving notebooks from local directory: /app
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Jupyter Server 2.14.2 is running at:
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] http://e29d05e13fd0:8888/jupyter/tree
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp]     http://127.0.0.1:8888/jupyter/tree
```

## Querying from Delta Lake
We connect to our datastores using the `DuckPond` class. DuckPond is a wrapper around a DuckDB connection with access to both our Metadata Database and our measurements stored in Delta Lake. The ability to query both sources of data from a single connection is useful for quickly accessing data for analysis.

There are two main ways to query data from Delta Lake:
1. Using the DuckPond `get_delta_data` method
2. Using the DuckPond connection to query directly

### Using the DuckPond `get_delta_data` method
DuckPond's `get_delta_data` method constructs a query based on the parameters you pass to it and returns a DuckDB DataFrame. It is useful for quickly accessing data for analysis. It takes the following optional parameters:
- `signal_names`: A string or list of signal names to query.
- `logger_ids`: A string or list of logger IDs to query.
- `animal_ids`: A string or list of animal IDs to query.
- `deployment_ids`: A string or list of deployment IDs to query.
- `recording_ids`: A string or list of recording IDs to query.
- `date_range`: A tuple of start and end dates to query.
- `limit`: The maximum number of rows to return.

The `get_delta_data` method returns a [DuckDB DuckDBPyConnection](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyConnection) which can be used to convert the data in many different formats including the following ([see documentation for a full list](https://duckdb.org/docs/api/python/conversion#result-conversion-duckdb-results-to-python))
- NumPy Array (`.fetchnumpy()`)
- Pandas DataFrame (`.df()`)
- Arrows Table (`.arrow()`)
- Polars DataFrame (`.pl()`)

Until a conversion method is called, the data is not loaded into memory. This allows for large queries to be run without using too much memory.

##### Example:

In [4]:
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

conn = duckpond.get_delta_data(    
    logger_ids="NL-02",
    animal_ids="mian-003",
    signal_names=["ECG_ICA2", "EEG_ICA5"],
)

display(conn)

┌─────────────┬─────────────────────────┬──────────────────────┐
│ signal_name │        datetime         │         data         │
│   varchar   │        timestamp        │        double        │
├─────────────┼─────────────────────────┼──────────────────────┤
│ EEG_ICA5    │ 2019-10-27 12:58:41     │   1.3214399938967718 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.002 │    2.956883955138856 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.004 │  -1.4821782253753728 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.006 │   -1.949447928587397 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.008 │   -4.052161593041506 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.01  │     3.42415365835088 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.012 │ -0.31400396734531255 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.014 │   -2.183082780193409 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.016 │   1.5550748455027839 │
│ EEG_ICA5    │ 2019-10-27 12:58:41.018 │   0.6205354390787357 │
│    ·        │           ·             │            ·         │
│    ·        │          

### Using the DuckPond connection to query directly
More complex queries can be run directly on the DuckPond connection. This is useful for queries that may not be supported by the `get_delta_data` method which has those involving grouping or aggregations. 

DuckDB runs sql very similar in syntax to other SQL databases. A full breakdown of the syntax can be found [in the documenation](https://duckdb.org/docs/sql/introduction).

The connection object can be found in the `duckpond.conn` attribute. To run queries, use the `sql` method which also returns a [DuckDB DuckDBPyConnection](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyConnection) which can be used to convert the data in many different formats including the following ([see documentation for a full list](https://duckdb.org/docs/api/python/conversion#result-conversion-duckdb-results-to-python))
- NumPy Array (`.fetchnumpy()`)
- Pandas DataFrame (`.df()`)
- Arrows Table (`.arrow()`)
- Polars DataFrame (`.pl()`)

##### Example:

In [16]:
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

df = duckpond.conn.sql(f"""
SELECT logger, signal_name, avg(data) as mean_data
FROM DeltaLake
GROUP BY logger, signal_name
""").df()
display(df)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,logger,signal_name,mean_data
0,NL-02,heading,-0.428566
1,NL-02,roll,0.203094
2,NL-02,GyrZ,0.004213
3,NL-02,MagY,-29.721872
4,NL-02,pitch,0.043052
5,NL-02,LEEG3_Raw_Ch8,-4.227085
6,NL-02,EEG_ICA5,-5.026906
7,NL-02,Pressure,0.052049
8,NL-02,ODBA,0.283147
9,NL-02,MagX,-77.693441


## Chaining Queries
Queries can be chained together to form a pipeline. This is useful for running complex queries that involve multiple steps.

##### Example:

In [17]:
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

# Get the filtered data
filtered_data = duckpond.get_delta_data(    
    logger_ids="NL-02",
    animal_ids="mian-003",
    signal_names=["ECG_ICA2", "EEG_ICA5"],
)

# Perform the aggregation on the filtered data
df = duckpond.conn.sql("""
SELECT signal_name, max(data) as mean_data
FROM filtered_data -- NOTE: filtered_data is our DuckDBPyConnection object from the previous query
GROUP BY signal_name
""").df()

display(df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,signal_name,mean_data
0,ECG_ICA2,12009.442710
1,EEG_ICA5,4788.966819


## Query Variables
Sometimes we don't want to hardcode variables in our queries. We can use the `execute` method to pass variables to the query.

##### Example:

In [24]:
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

signal_name = "ECG_ICA2"
df = duckpond.conn.execute(f"""
SELECT logger, signal_name, avg(data) as mean_data
FROM DeltaLake
WHERE signal_name = $1
GROUP BY logger, signal_name
""", [signal_name]).df()
display(df)

,logger,signal_name,mean_data
0,NL-02,ECG_ICA2,-3.549801


## Query Metadata Database
We can also query the Metadata Database directly. This is useful for querying data that is not stored in Delta Lake and joining it for queries on measurement data.

##### Example:

In [3]:
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

# Show all tables we have access to
print(duckpond.get_db_schema())

df = duckpond.conn.sql("""
SELECT avg(data) as mean_data
FROM DeltaLake 
JOIN Metadata.public.Animals ON DeltaLake.animal = Animals.id
WHERE Animals.project_id = 'test21_DozyDaisy'
AND signal_name = 'ECG_ICA2'
""").df()

display(df)

┌──────────┬─────────┬──────────────────────┬──────────────────────┬───────────────────────────────────────┬───────────┐
│ database │ schema  │         name         │     column_names     │             column_types              │ temporary │
│ varchar  │ varchar │       varchar        │      varchar[]       │               varchar[]               │  boolean  │
├──────────┼─────────┼──────────────────────┼──────────────────────┼───────────────────────────────────────┼───────────┤
│ Metadata │ public  │ Animal_Deployments   │ [id, animal_id, de…  │ [BIGINT, VARCHAR, VARCHAR]            │ false     │
│ Metadata │ public  │ Animals              │ [id, project_id, c…  │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR]  │ false     │
│ Metadata │ public  │ Deployments          │ [id, rec_date, ani…  │ [VARCHAR, DATE, VARCHAR, TIMESTAMP …  │ false     │
│ Metadata │ public  │ Files                │ [id, extension, ty…  │ [BIGINT, VARCHAR, VARCHAR, VARCHAR,…  │ false     │
│ Metadata │ public  │ Logger_Wi

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,mean_data
0,-3.549801


## Exporting Data to EDF
When it's easier to work with EDF files, we can export the data to an EDF file. This is useful for working with the data in other software packages.

The `create_mne_edf` function takes a DuckDB connection and a file path and creates an EDF file. 

*Note: it currently requires a lot of memory. Can be improved.*
*Note: it's lacking support for most info fields in the EDF file. Can be improved.*

##### Example:

In [2]:
from DiveDB.services.duck_pond import DuckPond
from DiveDB.services.utils.edf import create_mne_edf

duckpond = DuckPond()

conn = duckpond.get_delta_data(    
    logger_ids="NL-02",
    animal_ids="mian-003",
    signal_names=["ECG_ICA2", "EEG_ICA5"],
    limit=1000000,
)

create_mne_edf(conn, "test.edf")

Extracting EDF parameters from /app/test.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
EDF file created with 1 channels: test.edf


/usr/local/lib/python3.12/site-packages/pyedflib/edfwriter.py:124: UserWarning: Physical minimum for channel 0 (EEG_ICA5) is -3347.8341577782858, which has 19 chars, however, EDF+ can only save 8 chars, will be truncated to -3347.83, some loss of precision is to be expected
  warnings.warn('Physical minimum for channel {} ({}) is {}, which has {} chars, '\
/usr/local/lib/python3.12/site-packages/pyedflib/edfwriter.py:131: UserWarning: Physical maximum for channel 0 (EEG_ICA5) is 2382.9951172655838, which has 18 chars, however, EDF+ can only save 8 chars, will be truncated to 2382.995, some loss of precision is to be expected.
  warnings.warn('Physical maximum for channel {} ({}) is {}, which has {} chars, '\


<RawEDF | test.edf, 1 x 1000000 (2000.0 s), ~6 kB, data not loaded>

## Exporting Data to MNE Signal Array
For working with the data in MNE, we can export the data to an MNE Signal Array. This is useful for manipulating the data in MNE.

The `create_mne_array` function takes a DuckDB connection and returns an MNE RawArray.

##### Example:

In [7]:
from DiveDB.services.duck_pond import DuckPond
from DiveDB.services.utils.edf import create_mne_array

duckpond = DuckPond()

conn = duckpond.get_delta_data(    
    logger_ids="NL-02",
    animal_ids="mian-003",
    signal_names="ECG_ICA2",
    limit=1000000,
)

raw = create_mne_array(conn, resample=100, l_freq=1, h_freq=20)
display(raw)

Creating RawArray with float64 data, n_channels=1, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...  1999.998 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 331 samples (3.310 s)



<RawArray | 1 x 200000 (2000.0 s), ~1.5 MB, data loaded>